# Proyecto de predicion de rendimiento en estudiantes de 8 a 10 años


### Librerias
- Panda
- Sklearn

In [44]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


Datos de ejemplo

In [45]:

data = {
    "edad": [8, 9, 10, 8, 9, 10],
    "grado": [3, 4, 5, 3, 4, 5],
    "tipo_actividad": ["pregunta", "sopa_letras", "pregunta", "sopa_letras", "pregunta", "sopa_letras"],
    "dificultad": [1, 2, 2, 3, 2, 3],
    "tiempo_seg": [25, 120, 40, 200, 35, 180],
    "intentos": [1, 2, 1, 3, 2, 3],
    "pistas": [0, 1, 0, 0, 1, 0],
    "correcto": [1, 0, 1, 0, 1, 0],
    "nota": [9, 5, 8, 4, 7, 6],
    "palabras_encontradas": [0, 6, 0, 7, 0, 8],
    "palabras_totales": [1, 10, 1, 10, 1, 10],
    "evolucion_desempeno": [0.1, -0.2, 0.05, -0.3, 0.15, -0.1],
    "nivel_concentracion": [0.9, 0.6, 0.85, 0.5, 0.95, 0.7],
    "comparacion_historial": [0.05, -0.15, 0.1, -0.2, 0.12, -0.05]
}

df = pd.DataFrame(data)
print(df.head())

   edad  grado tipo_actividad  dificultad  tiempo_seg  intentos  pistas  \
0     8      3       pregunta           1          25         1       0   
1     9      4    sopa_letras           2         120         2       1   
2    10      5       pregunta           2          40         1       0   
3     8      3    sopa_letras           3         200         3       0   
4     9      4       pregunta           2          35         2       1   

   correcto  nota  palabras_encontradas  palabras_totales  \
0         1     9                     0                 1   
1         0     5                     6                10   
2         1     8                     0                 1   
3         0     4                     7                10   
4         1     7                     0                 1   

   evolucion_desempeno  nivel_concentracion  comparacion_historial  
0                 0.10                 0.90                   0.05  
1                -0.20                 0.60 

Preprocesado de datos

In [46]:

# =========================
# Codificar tipo_actividad
# =========================
le = LabelEncoder()
df["tipo_actividad"] = le.fit_transform(df["tipo_actividad"])

# =========================
# Calcular puntaje compuesto
# =========================
df["ratio_palabras"] = df["palabras_encontradas"] / df["palabras_totales"]
df["eficiencia_tiempo"] = 1 / df["tiempo_seg"]  # simple inverso del tiempo
df["puntaje_compuesto"] = (
    0.4 * (df["nota"]/10) + 
    0.3 * df["ratio_palabras"] + 
    0.1 * df["eficiencia_tiempo"] + 
    0.1 * (1 / df["intentos"]) + 
    0.1 * df["evolucion_desempeno"]
) * 100

print("Dataset con puntaje compuesto:")
print(df[["nota","ratio_palabras","eficiencia_tiempo","puntaje_compuesto"]])

X = df[["dificultad","tiempo_seg","intentos","pistas","ratio_palabras","eficiencia_tiempo","evolucion_desempeno","nivel_concentracion","comparacion_historial"]]
y = df["correcto"]



Dataset con puntaje compuesto:
   nota  ratio_palabras  eficiencia_tiempo  puntaje_compuesto
0     9             0.0           0.040000          47.400000
1     5             0.6           0.008333          41.083333
2     8             0.0           0.025000          42.750000
3     4             0.7           0.005000          37.383333
4     7             0.0           0.028571          34.785714
5     6             0.8           0.005556          50.388889


Entrenamiento de modelo

In [47]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model_log = LogisticRegression()
model_log.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


Evaluacion de modelo

In [48]:
nuevo_estudiante = pd.DataFrame([{
    "dificultad": 2,
    "tiempo_seg": 150,
    "intentos": 2,
    "pistas": 1,
    "ratio_palabras": 0.7,
    "eficiencia_tiempo": 1/150,
    "evolucion_desempeno": 0.1,
    "nivel_concentracion": 0.8,
    "comparacion_historial": 0.05
}])

# Probabilidad de éxito
probabilidad_exito = model_log.predict_proba(nuevo_estudiante)[0][1]
print("Probabilidad de éxito:", round(probabilidad_exito,2))

# =========================
# Recomendación adaptativa
# =========================
puntaje = (
    0.4 * 0.8 +  # nota estimada /10
    0.3 * 0.7 +  # ratio palabras
    0.1 * (1/150) +
    0.1 * (1/2) +
    0.1 * 0.1
) * 100

if puntaje < 50 or probabilidad_exito < 0.4:
    recomendaciones = ["Repetir sopas fáciles","Practicar preguntas básicas","Usar pistas si es necesario"]
elif puntaje < 75 or probabilidad_exito < 0.7:
    recomendaciones = ["Sopas de dificultad media","Preguntas de comprensión media","Revisar vocabulario"]
else:
    recomendaciones = ["Sopas difíciles","Preguntas complejas","Retos de análisis avanzado"]

print("Puntaje compuesto:", round(puntaje,2))
print("Recomendaciones:")
for r in recomendaciones:
    print("-", r)

Probabilidad de éxito: 0.02
Puntaje compuesto: 59.07
Recomendaciones:
- Repetir sopas fáciles
- Practicar preguntas básicas
- Usar pistas si es necesario


Evaluacion con nuevo estudiante

In [49]:
nuevo_estudiante = pd.DataFrame([{
    "edad": 9,
    "grado": 4,
    "tipo_actividad": 1,
    "dificultad": 2,
    "tiempo_seg": 150,
    "intentos": 2,
    "pistas": 1,
    "correcto": 1,
    "nota": 8,
    "secuencia_actividades": 1,
    "evolucion_desempeno": 0.1,
    "nivel_concentracion": 0.8,
    "comparacion_historial": 0.05
}])


print("Nivel estimado:", model.predict(nuevo_estudiante)[0])


Nivel estimado: Medio


### Creacion de un pipeline

Modelos a entrenar:
- Predecir el nivel de conocimiento
- Predecir el exito en una tarea

Resultado:


Modelos
- RandomForest
- LogicRegresion

In [102]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

Datos de ejemplo

In [103]:
df = pd.read_csv("/Users/km/Documents/VSPROJ/ML_Predidtion_rend/Performance_predition/datos/datos_actividades_1000.csv")

# ============================
# 2. Preprocesamiento
# ============================
le = LabelEncoder()
df["tipo_actividad"] = le.fit_transform(df["tipo_actividad"])

Modelo para predecir el nivel de compresion

In [104]:
# Crear variables dummy (binarias)
X = pd.get_dummies(df.drop("nivel_comprension", axis=1),  # eliminamos target
                   columns=["tipo_actividad"],            # columna a binarizar
                   drop_first=True)                        # evita multicolinealidad

# Target
y = df["nivel_comprension"]


# División entrenamiento-prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar Random Forest
modelo_compresion = RandomForestRegressor(n_estimators=300, random_state=42)
modelo_compresion.fit(X_train, y_train)

# Evaluar R²
r2 = modelo_compresion.score(X_test, y_test)
print("R²:", r2)


R²: 0.8427376865334442


Modelo para predecir tasa de exito en una tarea

In [105]:

# Creamos la variable "exito"
df["exito"] = ((df["nota"] >= 7) & (df["correcto"] == 1)).astype(int)
mask = np.random.rand(len(df)) < 0.05
df.loc[mask, "exito"] = 1 - df.loc[mask, "exito"]
# Convertir 'tipo_actividad' en variables binarias
X = pd.get_dummies(df.drop("exito", axis=1), columns=["tipo_actividad"], drop_first=True)
y = df["exito"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo_exito = LogisticRegression(max_iter=1000, C=1.0, solver='liblinear')
modelo_exito.fit(X_train, y_train)

# Probabilidad de éxito
probs = modelo_exito.predict_proba(X_test)[:,1]
print("Ejemplo de probabilidad de éxito:", probs[:5])

Ejemplo de probabilidad de éxito: [0.40510815 0.03308202 0.02241292 0.04498976 0.27848961]


/Users/km/Documents/VSPROJ/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/km/Documents/VSPROJ/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/km/Documents/VSPROJ/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [106]:
# Predicciones binarias
y_pred = modelo_exito.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC ROC:", roc_auc_score(y_test, probs))

Accuracy: 0.895
AUC ROC: 0.9446751206374143


/Users/km/Documents/VSPROJ/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/km/Documents/VSPROJ/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/km/Documents/VSPROJ/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Modelo de recomendacion segun las predicciones

In [116]:
def recomendar_actividad_dict(datos_estudiante, X, modelo_compresion, modelo_exito):
    nuevo_estudiante = pd.DataFrame([datos_estudiante])

    # Binarizar tipo_actividad
    if "tipo_actividad" in nuevo_estudiante.columns:
        nuevo_estudiante = pd.get_dummies(nuevo_estudiante, columns=["tipo_actividad"], drop_first=True)
    
    # Separar columnas para modelo_compresion (sin nivel_comprension)
    X_comp_modelo = X.drop(columns=["nivel_comprension"], errors='ignore')
    for col in X_comp_modelo.columns:
        if col not in nuevo_estudiante.columns:
            nuevo_estudiante[col] = 0
    nuevo_estudiante_comp = nuevo_estudiante[X_comp_modelo.columns]
    
    # Predicción comprensión
    comprension = modelo_compresion.predict(nuevo_estudiante_comp)[0]

    # Agregar nivel_comprension al DataFrame antes de predecir éxito
    nuevo_estudiante["nivel_comprension"] = comprension

    # Separar columnas para modelo_exito (X usado en entrenamiento de exito)
    X_exito_modelo = X.drop(columns=["exito"], errors='ignore')
    for col in X_exito_modelo.columns:
        if col not in nuevo_estudiante.columns:
            nuevo_estudiante[col] = 0
    nuevo_estudiante_exito = nuevo_estudiante[X_exito_modelo.columns]

    # Predicción probabilidad de éxito
    prob_exito = modelo_exito.predict_proba(nuevo_estudiante_exito)[:,1][0]

    # Recomendaciones
    recomendaciones = []
    if comprension < 0.4:
        recomendaciones.append("Actividades de repaso básico")
    elif comprension < 0.7:
        recomendaciones.append("Actividades intermedias con apoyo visual")
    else:
        recomendaciones.append("Actividades avanzadas con menos pistas")
    
    if prob_exito < 0.5:
        recomendaciones.append("Reducir dificultad o dar más pistas")
    else:
        recomendaciones.append("Incrementar dificultad progresivamente")
    
    return {
        "nivel_comprension": comprension,
        "prob_exito": prob_exito,
        "recomendaciones": recomendaciones
    }


In [ ]:
datos = {
    "edad": 10,
    "grado": 5,
    "dificultad": 3,
    "tiempo_seg": 200,
    "intentos": 2,
    "pistas": 1,
    "correcto": 1,
    "nota": 8,
    "secuencia_actividades": 2,
    "evolucion_desempeno": 0.1,
    "nivel_concentracion": 0.8,
    "comparacion_historial": 0.05,
    "tipo_actividad": "crucigrama"
}

resultado = recomendar_actividad_dict(datos, X, modelo_compresion, modelo_exito)   

print("Nivel de comprensión:", resultado["nivel_comprension"])
print("Probabilidad de éxito:", resultado["prob_exito"])
print("Recomendaciones:", resultado["recomendaciones"])



Nivel de comprensión: 0.6083405390384548
Probabilidad de éxito: 0.8125700959057147
Recomendaciones: ['Actividades intermedias con apoyo visual', 'Incrementar dificultad progresivamente']
